In [1]:
import sys; sys.path.append('../..')
import random
import pyzx as zx
import os
import pickle
import time
from pyzx.gflow import gflow
from pyzx.heuristics.gflow_calculation import focus_gflow, build_line, build_line_rec
import math
import copy
from fractions import Fraction

In [2]:
c = zx.Circuit.load('../../evaluation/feyn_bench_focused/before/adder_8_before').to_basic_gates()
g = c.to_graph()
zx.simplify.interior_clifford_simp(g)
# zx.simplify.spider_simp(g)
# zx.simplify.to_gh(g)
# zx.simplify.id_simp(g)
# zx.simplify.spider_simp(g)
g.normalize()
g = g.copy()
gf = gflow(g)
gff = focus_gflow(g, gf)
# zx.draw(g, labels=True)

spider_simp: 166. 125. 78. 44. 20. 16. 8. 6. 4.  9 iterations
id_simp: 51.  1 iterations
spider_simp: 21. 2.  2 iterations
pivot_simp: 32. 5. 1.  3 iterations
lcomp_simp: 6. 6.  2 iterations
id_simp: 4.  1 iterations
spider_simp: 3.  1 iterations
lcomp_simp: 2. 2.  2 iterations
id_simp: 1.  1 iterations
spider_simp: 1.  1 iterations


In [3]:
#zx.heuristics.gflow_calculation.check_focused_gflow(g,gff)

In [4]:
data = []
for out in g.outputs():
    line = build_line_rec(g,out, gff)
    print(line)
    data.append(line)
#     for elem in line:
#         totals.add(elem)
# print('totals: ')
# print(totals)

branching on vertex  392
branching on vertex  349
branching on vertex  347
branching on vertex  41
branching on vertex  39
[700, 393, 392, [33, 25, 3], [34, 438, 0], [384, 349, [33, 25, 3], [34, 438, 0], [348, 347, [33, 25, 3], [34, 438, 0], [339, 41, [33, 25, 3], [34, 438, 0], [40, 39, [32, 1], [33, 25, 3], [34, 438, 0]]]]]]
[701, 670, 669, 668, 660, 486, 485, 484, 33, 25, 3]
branching on vertex  652
branching on vertex  380
branching on vertex  345
branching on vertex  341
branching on vertex  37
[702, 695, 652, [33, 25, 3], [34, 438, 0], [431, 380, [33, 25, 3], [34, 438, 0], [346, 345, [33, 25, 3], [34, 438, 0], [344, 343, 342, 341, [33, 25, 3], [34, 438, 0], [340, 92, 91, 78, 38, 37, [33, 25, 3], [34, 438, 0], [36, 35, 34, 438, 0]]]]]]
branching on vertex  480
branching on vertex  472
branching on vertex  391
branching on vertex  380
branching on vertex  345
branching on vertex  341
branching on vertex  37
branching on vertex  390
branching on vertex  386
branching on vertex  380
b

In [5]:
gra = zx.heuristics.gflow_calculation.build_diagram_from_lines(g, data)

In [20]:
g.qubit(302)

21

In [24]:
# gra2 = gra.copy()
for v in g.vertices():
    gra.set_qubit(v, g.qubit(v))
    gra.set_row(v, g.row(v))
# gra2.normalize()
zx.draw(gra, labels=True)

In [21]:
gra2.qubit(302)

21

In [11]:
gra.neighbors(33)

dict_keys([25, 392, 349, 347, 41, 39, 484, 652, 380, 345, 341, 37, 480, 390, 386, 436, 434, 432])

In [14]:
gra.inputs()

(0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23)

In [16]:
gff[1][601]

{602, 616, 617, 619, 625, 640, 641, 643, 649, 682, 685, 687, 694, 713, 715}

In [17]:
g.neighbors(601)

dict_keys([529, 593, 602])

In [7]:
zx.draw(g, labels=True)

In [10]:
g.neighbors(0)

dict_keys([438])

In [4]:
gfocused = zx.heuristics.gflow_calculation.focus_gflow(g,gf)
zx.heuristics.gflow_calculation.check_focused_gflow(g,gfocused)

True

In [5]:
while True: #TODO: check when error occurrs during all simplification steps
    for vertex in g.vertices():
        if vertex in gf[1] and vertex not in g.inputs():
            wrong_vertex = False
            for neighbor in g.neighbors(vertex):
                if g.types()[neighbor] == 0:
                    wrong_vertex = True
            if wrong_vertex:
                continue
            for nu in zx.heuristics.neighbor_unfusion.get_possible_unfusion_neighbours(g, vertex, None, gf):
                gc = g.copy()
                gfc = gflow(gc)
    #             try:
    #                 zx.heuristics.neighbor_unfusion.unfuse_to_neighbor(gc, vertex, nu, Fraction(1,2))
                zx.heuristics.neighbor_unfusion.apply_lcomp(gc, (1.0, (vertex, list(g.neighbors(vertex))), nu), gfc[1])
    #             except:
    #                 import pdb
    #                 pdb.set_trace()
                if not gflow(gc):
                    print("error while unfusing ",vertex," to ",nu)

apply lcomp match  (1.0, (7, [5, 8]), 8)
unfuse to neighbor  7 8 61 60
apply lcomp match  (1.0, (7, [5, 8]), 5)
unfuse to neighbor  7 5 61 60
apply lcomp match  (1.0, (8, [7, 9, 10]), 10)
unfuse to neighbor  8 10 61 60
apply lcomp match  (1.0, (8, [7, 9, 10]), 7)
unfuse to neighbor  8 7 61 60
apply lcomp match  (1.0, (10, [8, 11]), 11)
unfuse to neighbor  10 11 61 60
apply lcomp match  (1.0, (10, [8, 11]), 8)
unfuse to neighbor  10 8 61 60
apply lcomp match  (1.0, (11, [6, 10, 12]), 12)
unfuse to neighbor  11 12 61 60
apply lcomp match  (1.0, (11, [6, 10, 12]), 10)
unfuse to neighbor  11 10 61 60
apply lcomp match  (1.0, (12, [11, 13]), 13)
unfuse to neighbor  12 13 61 60
apply lcomp match  (1.0, (12, [11, 13]), 11)
unfuse to neighbor  12 11 61 60
apply lcomp match  (1.0, (13, [9, 12, 14]), 14)
unfuse to neighbor  13 14 61 60
apply lcomp match  (1.0, (13, [9, 12, 14]), 12)
unfuse to neighbor  13 12 61 60
apply lcomp match  (1.0, (14, [13, 18]), 18)
unfuse to neighbor  14 18 61 60
apply

KeyboardInterrupt: 

In [6]:
gflow(g)[0]

{54: 0,
 55: 0,
 56: 0,
 57: 0,
 58: 0,
 30: 1,
 42: 1,
 50: 1,
 53: 1,
 59: 1,
 9: 2,
 49: 2,
 52: 2,
 48: 3,
 51: 3,
 34: 4,
 41: 4,
 47: 4,
 46: 5,
 45: 6,
 44: 7,
 43: 8,
 39: 9,
 40: 9,
 22: 10,
 29: 10,
 38: 10,
 37: 11,
 36: 12,
 35: 13,
 1: 14,
 33: 14,
 32: 15,
 31: 16,
 27: 17,
 28: 17,
 19: 18,
 26: 18,
 25: 19,
 24: 20,
 23: 21,
 2: 22,
 21: 22,
 20: 23,
 17: 24,
 18: 24,
 14: 25,
 16: 25,
 13: 26,
 15: 26,
 6: 27,
 12: 27,
 11: 28,
 10: 29,
 0: 30,
 8: 30,
 7: 31,
 3: 32,
 5: 32,
 4: 33}